<a href="https://colab.research.google.com/github/cccaaannn/deep_learning_colab/blob/master/efficientnet_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**test gpu**

In [0]:
!nvidia-smi

**imports**

In [0]:
import warnings
warnings.filterwarnings("ignore")

# keras and tf
import tensorflow as tf
import keras

# models
from keras.models import Sequential
from keras.models import Model

# backend
from keras import optimizers, metrics, models
import keras.backend as K

# layers
from keras.layers import Input, add, Conv2D, Flatten, Dense, Dropout, Activation
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D, BatchNormalization
from keras.layers.advanced_activations import LeakyReLU

# optimizers
from keras.optimizers import SGD

# regularizers
from keras.regularizers import l2

# training
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler

# save
import h5py

# keras aplications
from keras.applications import DenseNet201, DenseNet169, InceptionResNetV2, ResNet152V2, InceptionV3, DenseNet121, Xception, MobileNet, VGG19


# other libs
import math
import pickle
import os

**install efficieintNet**

In [0]:
pip install -U efficientnet

**import efficientNet**

In [0]:
import efficientnet.keras as efn 

**connect to the drive**

In [0]:
from google.colab import drive
drive.mount('/content/drive')

**set options**

In [0]:
image_size = 224 
batch_size = 8
nr_classes = 50
nr_ephocs = 75
model_path = "/content/drive/My Drive/efficientnet_106_class_model.hdf5"

**generate data**

In [0]:
train_datagen = ImageDataGenerator(
        # rotation -40 40
        rotation_range=40,

        # shift images and fill pixels with nearest
        width_shift_range=0.2,
        height_shift_range=0.2,
        fill_mode='nearest',

        horizontal_flip=True,
        zoom_range=0.3,

        shear_range=0.2,

        brightness_range=[0.1, 0.5],
        channel_shift_range=0.5,

        # normalization 
        featurewise_center=True,
        featurewise_std_normalization=True,
        # validation_split=0.2,
        )

valid_datagen = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True)

train = train_datagen.flow_from_directory("/content/drive/My Drive/data50/train", target_size=(image_size,image_size), batch_size=batch_size)
valid = valid_datagen.flow_from_directory("/content/drive/My Drive/data50/test", target_size=(image_size,image_size), batch_size=batch_size)

**build model**

In [0]:
base_model = efn.EfficientNetB7(include_top=False)

x=base_model.output
x=GlobalAveragePooling2D()(x)
preds=Dense(nr_classes,activation='softmax')(x)
model=Model(inputs=base_model.input,outputs=preds)

**or build model for transfer learning**

In [0]:
# create base model
# base_model = efn.EfficientNetB7(include_top=False, weights='imagenet')
base_model = efn.EfficientNetB7(include_top=False, weights='noisy-student')

# add the top
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(nr_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# freeze layers
for layer in base_model.layers:
    layer.trainable = False

# fix batchnormalization bug
for layer in base_model.layers:
    if isinstance(layer, BatchNormalization):
        layer.trainable = True
    else:
        layer.trainable = False

model.summary()

**or load model**

In [0]:
model = keras.models.load_model(model_path)

**set optimizer**

In [0]:
opt = SGD(lr=0.0001, momentum=0.9)

**compile model**

In [0]:
model.compile(loss = "categorical_crossentropy", optimizer = opt, metrics=['accuracy'])

**set checkpointer**

In [0]:
checkpointer = ModelCheckpoint(filepath= model_path, monitor='val_loss', save_best_only=True)

**train**

In [0]:
history = model.fit_generator(train, epochs=nr_ephocs, validation_data=valid, callbacks=[checkpointer])

**unfreeze model for full training**

In [0]:
nr_ephocs = 50
model_path = "/content/drive/My Drive/efficientnet_finetuned_106_class_model.hdf5"
for layer in model.layers:
    layer.trainable = True

**continue training**

In [0]:
history = model.fit_generator(train, epochs=nr_ephocs, validation_data=valid, callbacks=[checkpointer])